# BMock12 Data Analysis

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

The following metrics are computed for the complete and filtered set of assembled sequences, restricted to contigs of length above a specified minimum size: 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Maximum sequence length:** The length of the largest contig in the assembly.
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **Contiguity**
    - **Nx (where 0  < x  ⩽ 100):** Length for which the collection of all assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the assembly
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [25]:
ASSEMBLER_PROCESS_LIST = ["GATBMINIAPIPELINE", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "IDBA"]
PROCESS_TO_NAME = {"GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                   "SKESA": "SKESA",
                   "IDBA": "IDBA-UD"}

genomic_assemblers = ['SKESA', 'SPAdes', 'Unicycler']
metagenomic_assemblers = ['GATBMiniaPipeline', 'MEGAHIT', 'metaSPAdes']

best_min = ['Ns', 'contigs', 'filtered_Ns', 'filtered_contigs','misassembled contigs','misassembly events']
best_max = ['basepairs','filtered_basepairs','filtered_mapped_reads','filtered_n50','mapped_reads','max_contig','n50']

COLOURS = ['#5876c8', '#58AEC8', '#009392', '#39B185', '#9CCB86', '#E9E29C', '#EEB479', '#E88471', '#CF597E', '#a54765', '#a42a2a', '#835221', 'darkgray']

PLOT_SUBTITLES = ["2615840527", "2617270709", 
                  "2615840533 A", "2623620557", 
                  "2615840533 B", "2623620567", 
                  "2615840601", "2623620609", 
                  "2615840646", "2623620617", 
                  "2615840697", "2623620618", 
                  "2616644829"]

## Global metrics

In [38]:
report_glob = glob.glob('../Results/BMock12/report/pipeline_report_tables.json')
global_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for line in json_report[sample]['GlobalTable']:
                assembler = line['assembler']
                global_pipeline_metrics_df = global_pipeline_metrics_df.append({'run': stats_run,
                                                                                'sample': sample,
                                                                                'assembler': line['assembler'],
                                                                                'contigs': int(line['original']['contigs']),
                                                                                'basepairs': int(line['original']['basepairs']),
                                                                                'max_contig': int(line['original']['max_contig_size']),
                                                                                'n50': int(line['original']['N50']),
                                                                                'mapped_reads': line['original']['mapped_reads'],
                                                                                'Ns': int(line['original']['Ns']),
                                                                                'misassembled contigs': line['filtered']['misassembled_contigs'],
                                                                                'misassembly events': line['filtered']['misassembly_events'],
                                                                                'filtered_contigs': line['filtered']['contigs'],
                                                                                'filtered_basepairs': line['filtered']['basepairs'],
                                                                                'filtered_n50': line['filtered']['N50'],
                                                                                'filtered_Ns': line['filtered']['Ns'],
                                                                                'filtered_mapped_reads': line['filtered']['mapped_reads'],
                                                                               },
                                                                               ignore_index=True)
global_pipeline_metrics_df['type'] = np.where(global_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50', 'filtered_n50','misassembled contigs', 'misassembly events']] = global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','filtered_n50','misassembled contigs', 'misassembly events']].apply(pd.to_numeric)

Processing pipeline_report_tables.json data from BMock12...


In [4]:
global_pipeline_metrics_df

,Ns,assembler,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50,run,sample,type
0,0.0,GATBMiniaPipeline,50006704.0,8370.0,0.0,47507995.0,3982.0,89.057900,54313.0,96.623873,984007.0,4.0,8.0,48639.0,BMock12,SRR8073716,Metagenomic
1,0.0,IDBA-UD,48875595.0,8700.0,0.0,46615378.0,4070.0,91.045709,44194.0,96.675767,908381.0,17.0,29.0,38560.0,BMock12,SRR8073716,Metagenomic
2,0.0,MEGAHIT,49074360.0,2806.0,0.0,48470460.0,1513.0,95.973592,115470.0,96.767879,1037953.0,43.0,72.0,114742.0,BMock12,SRR8073716,Metagenomic
3,0.0,metaSPAdes,49062416.0,4251.0,0.0,48357131.0,1302.0,95.079017,122363.0,96.074584,1237485.0,3.0,5.0,118524.0,BMock12,SRR8073716,Metagenomic
4,0.0,SKESA,30258631.0,8612.0,0.0,27619442.0,2716.0,74.720173,34960.0,80.488520,451511.0,31.0,52.0,30082.0,BMock12,SRR8073716,Genomic
5,0.0,SPAdes,49402011.0,7501.0,0.0,47779053.0,2750.0,92.740629,76209.0,96.670037,1239293.0,5.0,9.0,67210.0,BMock12,SRR8073716,Genomic
6,0.0,Unicycler,37381909.0,3986.0,0.0,36255499.0,1328.0,89.476914,140943.0,93.820615,2778544.0,5.0,7.0,132926.0,BMock12,SRR8073716,Genomic


In [5]:
global_pipeline_metrics_df.to_csv("Tables/BMock12/Global metrics.csv")

## Breadth of coverage

In [10]:
_files = glob.glob('../Results/BMock12/results/*/stats/*_breadth_of_coverage_contigs.csv')

df_list = []
for f in _files:
    run = f.split('/')[-5]
    sample = f.split('/')[-3]
    assembler = f.split('/')[-1].split('_')[1]
    df = pd.read_csv(f)
    df['run'] = run
    df['Sample'] = sample
    df['Assembler'] = assembler
    df_list.append(df)

df = pd.concat(df_list)
df = df.dropna()

In [11]:
df

,Unnamed: 0,Reference,Breadth of Coverage,Contigs,run,Sample,Assembler
0,0,2615840527,0.996179,26,BMock12,SRR8073716,SPAdes
1,1,2615840533A,0.992550,24,BMock12,SRR8073716,SPAdes
2,2,2615840533B,0.992892,38,BMock12,SRR8073716,SPAdes
3,3,2615840601,0.968688,126,BMock12,SRR8073716,SPAdes
4,4,2615840646,0.984395,15,BMock12,SRR8073716,SPAdes
...,...,...,...,...,...,...,...
8,8,2623620557,0.887158,1159,BMock12,SRR8073716,IDBA-UD
9,9,2623620567,0.951104,951,BMock12,SRR8073716,IDBA-UD
10,10,2623620609,0.001396,2,BMock12,SRR8073716,IDBA-UD
11,11,2623620617,0.855941,610,BMock12,SRR8073716,IDBA-UD


In [35]:
fig=make_subplots(rows=7, cols=2, subplot_titles=PLOT_SUBTITLES,
                 shared_yaxes=True, shared_xaxes=True, x_title="<b>Contigs", y_title="<b>Breadth of Coverage")
row=1
col=1


for reference in sorted(df['Reference'].unique()):
    print('--' + reference)
    i=0
    print(row, col)
    for assembler in sorted(df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
        print('---' + assembler)
        showlegend=True if col==1 and row==1 else False
        
        contigs = list(df['Contigs'][(df['Assembler'] == assembler) & (df['Reference'] == reference)])
        boc = list(df['Breadth of Coverage'][(df['Assembler'] == assembler) & (df['Reference'] == reference)])
        if contigs: #list not empty
            for x, y in zip(contigs, boc):
                print(x, y)
                fig.add_trace(go.Scatter(x=[x],
                                         y=[y],
                                         name=assembler, marker=dict(color=COLOURS[i], size=12,opacity=0.5,
                                                                     line=dict(color='black',width=1)), 
                                         showlegend=False, opacity=1),
                              col=col, row=row)
        i+=1
    if row == 7:
        row = 1
        col += 1
    else:
        row += 1

fig.update_layout(plot_bgcolor='rgb(255,255,255)')
fig.update_layout(title=dict(text='Genome fragmentation variation <br><sup>BMock12 bacterial reference replicons</sup><br>',
                             x=0.5,
                             y=0.97,
                             xanchor='center',
                             yanchor='top'),
                 font=dict(size=18))
fig.update_xaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC', rangemode='tozero', type='log', range=[0,3])
fig.update_yaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC', range=[0, 1])

for i in fig['layout']['annotations']:
    i['font']['size'] = 18

fig.update_layout(xaxis_showticklabels=True, xaxis2_showticklabels=True, xaxis3_showticklabels=True, xaxis4_showticklabels=True, xaxis5_showticklabels=True,  xaxis6_showticklabels=True,  xaxis7_showticklabels=True,  xaxis8_showticklabels=True,
                  xaxis9_showticklabels=True, xaxis10_showticklabels=True, xaxis11_showticklabels=True, xaxis12_showticklabels=True)
fig.update_layout(yaxis_showticklabels=True, yaxis2_showticklabels=True, yaxis3_showticklabels=True, yaxis4_showticklabels=True, yaxis5_showticklabels=True,  yaxis6_showticklabels=True,  yaxis7_showticklabels=True,  yaxis8_showticklabels=True,
                  yaxis9_showticklabels=True, yaxis10_showticklabels=True, yaxis11_showticklabels=True, yaxis12_showticklabels=True)


# just for display purpose, create traces so that legend contains colors.  does not connect graph

legend_trace = [go.Bar(name=assembler, x=[-10], marker_color=c, showlegend=True,  legendgroup="Assemblers", legendgrouptitle_text="Assembler")
        for assembler,c in zip(sorted(df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True),COLOURS)]
    
fig.update_traces(showlegend=False).add_traces(legend_trace)

fig.show()
plot(fig, filename='Plots/BMock12/Genome fragmentation.html', auto_open=False)

--2615840527
1 1
---Unicycler
20 0.9955483640279542
---SPAdes
26 0.9961786275281844
---SKESA
47 0.98794624534085
---metaSPAdes
17 0.9974071543950792
---MEGAHIT
26 0.997277762550661
---IDBA-UD
30 0.995130414457382
---GATBMiniaPipeline
27 0.9948643959557396
--2615840533A
2 1
---Unicycler
23 0.99198226394391
---SPAdes
24 0.9925504269031996
---SKESA
37 0.9632714862244888
---metaSPAdes
22 0.995157923920832
---MEGAHIT
26 0.994355466957712
---IDBA-UD
28 0.9763655923635776
---GATBMiniaPipeline
27 0.9891394966974308
--2615840533B
3 1
---Unicycler
35 0.9893313792730468
---SPAdes
38 0.9928916486774292
---SKESA
117 0.9731816905363662
---metaSPAdes
38 0.9935304760787576
---MEGAHIT
34 0.9868611281651284
---IDBA-UD
58 0.986183453151557
---GATBMiniaPipeline
34 0.989016281973743
--2615840601
4 1
---Unicycler
119 0.9673770192820992
---SPAdes
126 0.9686877229840054
---SKESA
184 0.9437483464547206
---metaSPAdes
124 0.9702972389511866
---MEGAHIT
135 0.970121121702498
---IDBA-UD
153 0.9683601449014448
---GA

'Plots/BMock12/Genome fragmentation.html'

## SNP analysis

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Load data

In [42]:
report_glob = glob.glob('../Results/BMock12/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                        'sample': sample,
                                                                                        'assembler': item['assembler'],
                                                                                        'reference': reference,
                                                                                        'LSA': item['contiguity'],
                                                                                        'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                        'multiplicity': item['multiplicity'],
                                                                                        'validity': item['validity'],
                                                                                        'parsimony': item['parsimony'],
                                                                                        'identity': item['identity'],
                                                                                        'lowest_identity': item['lowest_identity'],
                                                                                        'L90': item['L90'],
                                                                                        'contigs': item['aligned_contigs'],
                                                                                        'NA50': item['NA50'],
                                                                                        'NG50': item['NG50'],
                                                                                        'basepairs': item['aligned_basepairs'],
                                                                                        'Ns': item['Ns'],
                                                                                        'misassembled contigs': item['misassembled_contigs'],
                                                                                        'misassembly events': item['misassembly_events'],
                                                                                        'snps': item['snps']},
                                                                                       ignore_index=True)

reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events', 'multiplicity','validity','parsimony','identity','lowest_identity']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events','multiplicity','validity','parsimony','identity','lowest_identity']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

Processing pipeline_report_tables.json data from BMock12...


,L90,LSA,NA50,NG50,Ns,assembler,basepairs,breadth_of_coverage,contigs,identity,...,misassembled contigs,misassembly events,multiplicity,parsimony,reference,run,sample,snps,validity,type
0,8.0,0.273812,914828.0,914828.0,0.0,GATBMiniaPipeline,3575279.0,0.994864,27.0,0.999675,...,0.0,0.0,0.997753,0.997753,2615840527,BMock12,SRR8073716,18.0,1.000000,Metagenomic
1,9.0,0.252768,472675.0,472675.0,0.0,IDBA-UD,3576235.0,0.995130,30.0,0.999045,...,0.0,0.0,0.997020,0.997020,2615840527,BMock12,SRR8073716,57.0,1.000000,Metagenomic
2,7.0,0.288938,914947.0,914947.0,0.0,MEGAHIT,3583952.0,0.997278,26.0,0.982529,...,1.0,1.0,0.996371,0.996669,2615840527,BMock12,SRR8073716,369.0,0.999701,Metagenomic
3,5.0,0.344345,1059355.0,1059355.0,0.0,metaSPAdes,3584417.0,0.997407,17.0,0.998740,...,0.0,0.0,0.997281,0.997281,2615840527,BMock12,SRR8073716,272.0,1.000000,Metagenomic
4,19.0,0.125638,158894.0,158894.0,0.0,SKESA,3550417.0,0.987946,47.0,1.000000,...,0.0,0.0,0.998398,0.998398,2615840527,BMock12,SRR8073716,0.0,1.000000,Genomic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,154.0,0.040115,35730.0,33862.0,0.0,MEGAHIT,4023422.0,0.970799,306.0,0.989338,...,3.0,3.0,1.001127,1.008166,2623620618,BMock12,SRR8073716,2298.0,0.993018,Metagenomic
87,212.0,0.023310,22076.0,22076.0,0.0,metaSPAdes,4030931.0,0.972611,370.0,0.991704,...,0.0,0.0,1.035639,1.037884,2623620618,BMock12,SRR8073716,9915.0,0.997837,Metagenomic
88,0.0,0.008835,6643.0,1298.0,0.0,SKESA,2241972.0,0.540959,511.0,0.999993,...,0.0,0.0,0.998582,0.998582,2623620618,BMock12,SRR8073716,18.0,1.000000,Genomic
89,0.0,0.012836,13464.0,10524.0,0.0,SPAdes,3697973.0,0.892273,593.0,0.998705,...,0.0,0.0,1.004710,1.004796,2623620618,BMock12,SRR8073716,1182.0,0.999914,Genomic


In [43]:
reference_pipeline_metrics_df.to_csv("Tables/BMock12/Reference metrics.csv")

In [54]:
reference_pipeline_metrics_df[reference_pipeline_metrics_df.type=="Metagenomic"].describe()

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,snps,validity
count,52.000000,52.000000,5.200000e+01,5.200000e+01,52.0,5.200000e+01,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,89.846154,0.075765,1.570732e+05,1.553890e+05,0.0,3.907009e+06,0.890501,221.673077,0.979834,0.716057,1.788462,2.692308,0.953243,0.960790,7686.000000,0.992894
std,186.079509,0.077557,2.334567e+05,2.338411e+05,0.0,1.848434e+06,0.263008,294.400735,0.040644,0.311264,3.582838,5.300901,0.160178,0.163806,15624.744925,0.021537
min,0.000000,0.000248,1.138000e+03,0.000000e+00,0.0,9.453000e+03,0.001396,2.000000,0.822403,0.008278,0.000000,0.000000,0.342315,0.342315,0.000000,0.891972
25%,8.750000,0.023181,2.220125e+04,2.197075e+04,0.0,3.339794e+06,0.954920,29.500000,0.985606,0.606929,0.000000,0.000000,0.990717,0.990717,304.250000,0.997651
50%,36.000000,0.051088,8.761100e+04,8.357850e+04,0.0,4.027176e+06,0.975568,128.000000,0.992700,0.835786,0.000000,0.000000,0.997002,0.997634,820.500000,0.999824
75%,69.750000,0.107893,1.231582e+05,1.262235e+05,0.0,4.493090e+06,0.989519,208.000000,0.998798,0.957370,1.250000,2.250000,1.000370,1.006252,5612.250000,1.000000
max,1044.000000,0.344345,1.059355e+06,1.059355e+06,0.0,7.136418e+06,0.999828,1186.000000,0.999982,0.999519,14.000000,22.000000,1.035639,1.115585,70196.000000,1.000000
